# Segmenting and Clustering Neighbourhoods in Toronto - Part 03
## Applied Data Science Capstone 
### IBM Data Science Professional Certificate

In [1]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import folium, requests
from   geopy.geocoders import Nominatim
from   sklearn.cluster import KMeans

### Load in the dataframe that was saved in the previous notebook

I'll only consider boroughs that have the "Toronto" in the name, so create a new dataframe as a subset of the original neighborhoods dataframe

In [2]:
neighborhoods = pd.read_csv("neighbourhoods.csv")
torontoData   = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)

torontoData.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Explore and cluster the neighborhoods in Toronto

Use Nominatim to get the latitude and longitude of Toronto for later use in the folium map

In [3]:
address    = 'Toronto'
geolocator = Nominatim(user_agent="TorontoExplorer")
toronto    = geolocator.geocode(address)
latitude   = toronto.latitude
longitude  = toronto.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Credentials for using the Foursquare API

In [4]:
CLIENT_ID     = "HIIHGUND2MSZDULMWUO213NKCZHP0U2DNMLYVPWSWR25MZND" # My Foursquare ID
CLIENT_SECRET = "OCIZI2HV4RUH2XCOEQYCX5CGQPGGCHWTPTOAFX4LSBZMESGY" # My Foursquare Secret
VERSION       = "20180605"                                         # Foursquare API version

print("My credentails:")
print("CLIENT_ID:    " + CLIENT_ID)
print("CLIENT_SECRET:" + CLIENT_SECRET)

My credentails:
CLIENT_ID:    HIIHGUND2MSZDULMWUO213NKCZHP0U2DNMLYVPWSWR25MZND
CLIENT_SECRET:OCIZI2HV4RUH2XCOEQYCX5CGQPGGCHWTPTOAFX4LSBZMESGY


Use the function defined in the lab to generate a dataframe of venues nearby the latitude and longitude coordinates for a given neighbourhood

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
torontoVenues = getNearbyVenues(names     =torontoData["Neighborhood"],
                                latitudes =torontoData["Latitude"],
                                longitudes=torontoData["Longitude"])
torontoVenues.head()

The Beaches


KeyError: 'groups'

In [ ]:
torontoVenues.groupby("Neighborhood").count()
print("There are {} uniques categories.".format(len(torontoVenues["Venue Category"].unique())))

Apply one hot encoding for the venue categories

In [ ]:
# one hot encoding
torontoOnehot = pd.get_dummies(torontoVenues[["Venue Category"]], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoOnehot["Neighborhood"] = torontoVenues["Neighborhood"] 

# move neighborhood column to the first column
fixedColumns  = [torontoOnehot.columns[-1]] + list(torontoOnehot.columns[:-1])
torontoOnehot = torontoOnehot[fixedColumns]

torontoOnehot.head()

Group the rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
torontoGrouped = torontoOnehot.groupby("Neighborhood").mean().reset_index()
torontoGrouped

Print each neighborhood along with the top 5 most common venues

In [ ]:
numTopVenues = 5

for hood in torontoGrouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torontoGrouped[torontoGrouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(numTopVenues))
    print('\n')

Create a function to sort the venues in descending order

In [ ]:
def mostCommonVenues(row, numTopVenues):
    rowCategories       = row.iloc[1:]
    rowCategoriesSorted = rowCategories.sort_values(ascending=False)
    
    return rowCategoriesSorted.index.values[0:numTopVenues]

Create a new dataframe and display the top 10 venues for each neighborhood

In [ ]:
numTopVenues = 10
indicators   = ["st", "nd", "rd"]

# create columns according to number of top venues
columns = ["Neighborhood"]
for ind in np.arange(numTopVenues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoodsVenuesSorted                 = pd.DataFrame(columns=columns)
neighborhoodsVenuesSorted["Neighborhood"] = torontoGrouped["Neighborhood"]

for ind in np.arange(torontoGrouped.shape[0]):
    neighborhoodsVenuesSorted.iloc[ind, 1:] = mostCommonVenues(torontoGrouped.iloc[ind, :], numTopVenues)

neighborhoodsVenuesSorted.head()

### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [ ]:
numClusters = 10

torontoGroupedClustering = torontoGrouped.drop("Neighborhood", 1)

# run k-means clustering
kmeans = KMeans(n_clusters=numClusters, random_state=0).fit(torontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
neighborhoodsVenuesSorted.insert(0, "Cluster Labels", kmeans.labels_)

torontoMerged = torontoData

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torontoMerged = torontoMerged.join(neighborhoodsVenuesSorted.set_index("Neighborhood"), on="Neighborhood")

torontoMerged.head() # check the last columns!

Create a folium map and visualize the resulting clusters

In [48]:
# create map
mapClusters = folium.Map(location=[toronto.latitude, toronto.longitude], zoom_start=11)

# set color scheme for the clusters
x          = np.arange(numClusters)
ys         = [i + x + (i*x)**2 for i in range(numClusters)]
colorArray = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow    = [colors.rgb2hex(i) for i in colorArray]

# add markers to the map
markerColours = []
for lat, lon, poi, cluster in zip(torontoMerged["Latitude"], torontoMerged["Longitude"], torontoMerged["Neighborhood"], torontoMerged["Cluster Labels"]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapClusters)
       
mapClusters

One observation is that all of the neighborhoods around the centre of Toronto appear to be very similar. I experimented with 2, 5, and 10 clusters and found that when there is a small 'pocket' just north of the CBD which appears unique. With 5 clusters the neighborhoods to the north and the east appear quite different, and with 10 clusters, neighborhoods to the south west appear different. 